In [6]:
import torch
import torch.nn as nn
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
from torchvision.models import efficientnet_b0

class BinaryEfficientNet(nn.Module):
    def __init__(self, pretrained=True):
        super(BinaryEfficientNet, self).__init__()
        
        # Load pre-trained EfficientNet-B0
        self.base_model = efficientnet_b0(pretrained=pretrained)
        
        # Get number of features in the final layer
        num_features = self.base_model.classifier[1].in_features
        
        # Replace the classifier with a binary classifier
        self.base_model.classifier = nn.Sequential(
            nn.Dropout(p=0.2, inplace=True),
            nn.Linear(num_features, 2)
        )

    def forward(self, x):
        return self.base_model(x)

In [7]:
from torchsummary import summary
import torch

model = BinaryEfficientNet()
summary(model, input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
              SiLU-3         [-1, 32, 112, 112]               0
            Conv2d-4         [-1, 32, 112, 112]             288
       BatchNorm2d-5         [-1, 32, 112, 112]              64
              SiLU-6         [-1, 32, 112, 112]               0
 AdaptiveAvgPool2d-7             [-1, 32, 1, 1]               0
            Conv2d-8              [-1, 8, 1, 1]             264
              SiLU-9              [-1, 8, 1, 1]               0
           Conv2d-10             [-1, 32, 1, 1]             288
          Sigmoid-11             [-1, 32, 1, 1]               0
SqueezeExcitation-12         [-1, 32, 112, 112]               0
           Conv2d-13         [-1, 16, 112, 112]             512
      BatchNorm2d-14         [-1, 16, 1

In [8]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # EfficientNet expects 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Define dataset paths
train_dir = "/Users/emmanuelgeorgep/Desktop/tempp1/Dataset/Train"
test_dir = "/Users/emmanuelgeorgep/Desktop/tempp1/Dataset/Test"

# Load datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Print dataset info
print("Train samples:", len(train_dataset))
print("Test samples:", len(test_dataset))
print("Classes:", train_dataset.classes)

Train samples: 12000
Test samples: 2000
Classes: ['Blocked', 'Normal']


In [9]:
def train_model(model, train_loader, criterion, optimizer, device, epochs=5):
    model.to(device)
    model.train()

    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)             
            loss = criterion(outputs, labels)     
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        accuracy = correct / total
        avg_loss = running_loss / len(train_loader)
        print(f"Epoch [{epoch+1}/{epochs}] | Loss: {avg_loss:.4f} | Accuracy: {accuracy:.4f}")

In [10]:
device = torch.device("mps" if torch.cuda.is_available() else "cpu")
model = BinaryEfficientNet(pretrained=True)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
train_model(model, train_loader, criterion, optimizer, device, epochs=5)

Epoch [1/5] | Loss: 0.0683 | Accuracy: 0.9845
Epoch [2/5] | Loss: 0.0116 | Accuracy: 0.9965
Epoch [2/5] | Loss: 0.0116 | Accuracy: 0.9965


KeyboardInterrupt: 

In [11]:
def test_model(model, test_loader, criterion, device):
    model.to(device)
    model.eval()  # Set model to evaluation mode

    total_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)             # [B, 2]
            loss = criterion(outputs, labels)  # CrossEntropyLoss

            total_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    avg_loss = total_loss / len(test_loader)
    accuracy = correct / total

    print(f"\nTest Loss: {avg_loss:.4f} | Test Accuracy: {accuracy:.4f}")
    return avg_loss, accuracy
    
test_model(model, test_loader, criterion, device)


Test Loss: 0.0295 | Test Accuracy: 0.9880


(0.029496028589150344, 0.988)

In [ ]:
from IPython.display import FileLink

torch.save(model.state_dict(), "EfficientNet-KL.pth")
